In [1]:
import cv2
import dlib
import numpy as np
import matplotlib.pyplot as plt

from scipy.spatial import distance as dist

from io import BytesIO
from IPython.display import clear_output, Image, display
from PIL import Image as Img

In [2]:
FACE = list(range(17, 68))
FACE_COMPLETA = list(range(0, 68))
LABIO = list(range(48, 61))
SOMBRANCELHA_DIRETA = list(range(17, 22))
SOMBRANCELHA_ESQUERDA = list(range(22, 27))
OLHO_DIREITO = list(range(36,42))
OLHO_ESQUERDO = list(range(42,48))
NARIZ = list(range(27,35))
MANDIBULA = list(range(0,17))

In [3]:
def padronizar_imagem(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (500, 400))
    return frame

In [4]:
classificador_dlib_68_path = "classifiers/shape_predictor_68_face_landmarks.dat"
classificador_dlib = dlib.shape_predictor(classificador_dlib_68_path)
detector_face = dlib.get_frontal_face_detector()

In [5]:
def pontos_marcos_faciais(imagem):
    
    retangulos = detector_face(imagem, 1)
    
    if len(retangulos) == 0:
        return None
    
    marcos = []
    
    for ret in retangulos:
        marcos.append(np.matrix([[p.x ,p.y] for p in classificador_dlib(imagem, ret).parts()]))
        
        return marcos

In [6]:
def exibir_video(frame):
    img = Img.fromarray(frame, 'RGB')
    buffer = BytesIO()
    img.save(buffer, format='JPEG')
    display(Image(data=buffer.getvalue()))
    clear_output(wait=True)

In [7]:
def aspecto_razao_boca(pontos_boca):
    a = dist.euclidean(pontos_boca[3], pontos_boca[9])
    b = dist.euclidean(pontos_boca[2], pontos_boca[10])
    c = dist.euclidean(pontos_boca[4], pontos_boca[8])
    d = dist.euclidean(pontos_boca[0], pontos_boca[6])
    
    aspecto_razao = (a + b + c)/(3.0*d)
    return aspecto_razao

In [11]:
def anotar_marcos_casca_convexa(imagem, marcos):
    retangulos = detector_face(imagem, 1)
    
    if len(retangulos) == 0:
        return None
    
    for idx, ret in enumerate(retangulos):
        marco = marcos[idx]
        
        pontos = cv2.convexHull(marco[OLHO_ESQUERDO])
        cv2.drawContours(imagem, [pontos], 0, (0,255,0), 2)
        
        pontos = cv2.convexHull(marco[OLHO_DIREITO])
        cv2.drawContours(imagem, [pontos], 0, (0,255,0), 2)
    
    return imagem

In [9]:
def aspecto_razao_olhos(pontos_olhos):
    
    a = dist.euclidean(pontos_olhos[1], pontos_olhos[5])
    b = dist.euclidean(pontos_olhos[2], pontos_olhos[4])
    c = dist.euclidean(pontos_olhos[0], pontos_olhos[3])
    
    aspecto_razao = (a + b)/(2.0 * c)
    
    return aspecto_razao

In [14]:
min_olho_esq = 1
min_olho_dir = 1
video = cv2.VideoCapture(0)

try:
    while(True):
        capture_ok, frame = video.read()
        if capture_ok:
            frame = padronizar_imagem(frame)
            
            marcos_faciais = pontos_marcos_faciais(frame)
            
            if marcos_faciais is not None:
                ar_olho_esq = aspecto_razao_olhos(marcos_faciais[0][OLHO_ESQUERDO])
                ar_olho_dir = aspecto_razao_olhos(marcos_faciais[0][OLHO_DIREITO])
                
                ar_olho_esq = round(ar_olho_esq, 3)
                ar_olho_dir = round(ar_olho_dir, 3)
                
                if ar_olho_esq < min_olho_esq:
                    min_olho_esq = ar_olho_esq
                    
                if ar_olho_dir < min_olho_dir:
                    min_olho_dir = ar_olho_dir
                    
                info_olho_esq = 'olho esquerdo ' + str(ar_olho_esq) + ' minimo ' + str(min_olho_esq)
                info_olho_dir = 'olho direito ' + str(ar_olho_dir) + ' minimo ' + str(min_olho_dir)
                
                frame = anotar_marcos_casca_convexa(frame, marcos_faciais)
                
                cv2.putText(frame, info_olho_esq, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
                cv2.putText(frame, info_olho_dir, (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
            
            exibir_video(frame)

except KeyboardInterrupt:
    video.release()
    print('Interrompido')

Interrompido


In [13]:
video.release()